In [7]:
import os
from itertools import permutations,combinations

In [8]:
template = """
FROM prolocutor/docker-python-sqlite:3.12
#FROM 229931fb6377 #testing 
RUN cd /tmp/sqlite-autoconf-3120200 \                                                                    
        && ./configure --prefix=/usr --disable-static \                                                  
                CFLAGS="-g {{ flag -}}" \
        && make \                                                                                        
        && make install \                                                                                
        && rm -r /tmp/*                                                                                  

CMD ["python3"]

"""
versionroot = "./sqlite3.12/"

readmetemplate = """
# SQLITE 3.12
## Available Dockerfiles:
{% for d in dockerfiles %}
{{d}}
{% endfor %}
"""

with open(versionroot+"README.jinja2","w") as f:
    f.write(readmetemplate)
with open(versionroot+"Dockerfile.jinja2","w") as f:
    f.write(template)

In [9]:
tree = {"sqlite3.12":"./sqlite3.12"}

trunk = {"Os":"-Os",
         "O2":"-O2",
        "debug":"-DSQLITE_DEBUG"}

available_flags = {"fts5":"-DSQLITE_ENABLE_FTS5",
                   "rtree":"-DSQLITE_ENABLE_RTREE",
                   "column-metadata":"-DSQLITE_ENABLE_COLUMN_METADATA",
                   "json1":"-DSQLITE_ENABLE_JSON1"}

In [10]:
dockerfile_dir = []
for root in tree.values():
    for p in permutations(available_flags):
        for item in trunk:
            d = os.path.join(versionroot,item,"/".join(p))
            try: os.makedirs(d)
            except: pass
            dockerfile_dir.append(d)

In [12]:
flag_combinations = []
for L in range(0, len(available_flags)+1):
    for subset in combinations(available_flags, L):
        for k,v in trunk.items():
            flag_combinations.append(sorted(subset+(k,)))

In [13]:
all_flags = available_flags.update(trunk)

In [18]:
from jinja2 import Template, Environment, FileSystemLoader
import collections

#Everything is a template
obreros_env = Environment(loader=FileSystemLoader('.'))
link = 'https://github.com/disarticulate/docker-python-sqlite/blob/master'
dockerfiles = []

for k,v in tree.items():
    for root, subdirs, files in os.walk(v):
        root_dirs = root.replace(v,'')
        root_flags = root.replace(v,'').split('/')[1:]
        
        for flags in flag_combinations:
            if collections.Counter(root_flags) == collections.Counter(flags):
                flag = " ".join([available_flags[flag] for flag in flags])
                obreros_template = obreros_env.get_template('Dockerfile.jinja2').render(flag=flag)
                Dockerfile = root+"/"+"Dockerfile"
                dockerfiles.append("["+Dockerfile[1:]+"]("+link+Dockerfile[1:]+")")
                with open(Dockerfile, "w") as f:
                    f.write(obreros_template)
                
obreros_template = obreros_env.get_template(versionroot+'README.jinja2').render(dockerfiles=dockerfiles)

with open(versionroot+'README.md', "w") as f:
    f.write(obreros_template)